# Construction d'un système de recommandation

Nous avons décidé d'orienter notre projet sur la recommendation de films.
En effet durant ce confinement, nous avons eu le temps de visionner beaucoup de films,
mais nous nous sommes rendus compte que nous passions quasiment autant de temps
à choisir le film qu'à le regarder. D'où la nécessité de créer un système de re-
commendations afin d'optimiser notre temps de visionnage.
Nous avons chercher une base de données assez exploitable afin de mener à bien
notre projet. Nous nous sommes basés sur la base de données de 'The Movies Dataset'.


In [2]:
import numpy as np
import pandas as pd
import math
import re

## Fetching and cleaning data

Nous utilisons deux tables de données. L'une, *movies_metadata.csv*, contient une liste de films et des informations relativesau genre, date de sortie etc. 

### Informations sur les films

In [3]:
movies = pd.read_csv("movies_metadata.csv")
movies.head()

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [5]:
def filter_correct_id(word):
    if re.fullmatch(r'[0-9]+', word):
        return word
    return "wrong_id"

In [6]:
# don't re-run
movies = movies[~movies.id.duplicated()]
movies.id = movies.id.apply(filter_correct_id)
movies = movies[movies.id != "wrong_id"]
movies.id = movies.id.astype('int64')

### Avis des utilisateurs

In [8]:
ratings = pd.read_csv("ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [9]:
# ne pas re-run !
ratings = ratings.drop(columns=['timestamp'])
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [10]:
ratings[(ratings['userId'] == 1) & (ratings['movieId'] == 31)]

,userId,movieId,rating
0,1,31,2.5


In [11]:
print(min(ratings.rating), max(ratings.rating))
ratings.describe()
ratings.dtypes

0.5 5.0


userId       int64
movieId      int64
rating     float64
dtype: object

In [13]:
nbPers = len(ratings.userId.unique())
nbMovi = len(ratings.movieId.unique())

## User Based

Pour ce système, nous n'avons que besoin des notations des utilisateurs et des titres des films associés. Nous allons translater les notes afin que la moyenne des notes pour chaque utilisateur se trouve à 0. Par abus de langage nous appelons ces nouvelles notes les notes *normalisées*. 

In [24]:
movies_title = movies.loc[movies['id'].isin(ratings.movieId), ['id', 'title']]
print(len(ratings.movieId.unique()))
ratings_small = ratings.loc[ratings['userId'] <= 30]

9066


In [21]:
def mean_rating(uid):
    '''
    Retourne la moyenne des notes données par l'utilisateur d'id uid
    '''
    n = ratings.loc[ratings['userId'] == uid].count().loc['userId']
    s = ratings.loc[ratings['userId'] == uid].sum().loc['rating']
    return s / n

In [22]:
def normalize(df):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées
    '''
    mean = ratings.loc[:, ['userId']].drop_duplicates()
    mean['mu'] = mean['userId'].map(lambda uid : mean_rating(uid))
    mean = mean.set_index('userId')
    df['rating_norm'] = df[['userId', 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row['userId'])]['mu'], axis=1)

In [33]:
normalize(ratings_small)
print(ratings_small.head())

   userId  movieId  rating  rating_norm
0       1       31     2.5        -0.05
1       1     1029     3.0         0.45
2       1     1061     3.0         0.45
3       1     1129     2.0        -0.55
4       1     1172     4.0         1.45


/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Regrouper les utilisateurs dans des peer-group

Pour déterminer si deux utilisateurs se ressemblent en termes de goûts, nous utilisons un taux de corrélation sur les avis données. Nous allons comparer quatres taux de corrélations différents. Le premier ```cor()``` calcule le taux de corrélation classique donné par la formule :

Le taux de corrélation ajusté ```cor_adj()``` permet de ne pas donner trop d'importance aux films populaires que beaucoup de personnes ont vu.

Le taux de correlation calculé par ```cor_dis()``` permet de ne pas donner une correlation trop élevée si les deux utilisateurs n'ont pas donné assez d'avis sur des films en commun. 

Enfin la fonction ```cor_dis_adj()``` fait un mélange des deux dernières amélioration : il filtre les films trop populaire et n'apporte de l'importance seulement si deux personnes ont données leur avis sur un certain nombre de films.

In [15]:
def cor(u, v, df):
    Iu = df.loc[df['userId'] == u, ['movieId']]
    Iv = df.loc[df['userId'] == v, ['movieId']]
    Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')

    su = df.loc[(df['userId'] == u) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    sv = df.loc[(df['userId'] == v) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    su = np.array(su)
    sv = np.array(sv)
    
    return np.dot(su, sv) / math.sqrt(np.dot(su, su) * np.dot(sv, sv))

In [93]:
def cor_adj(u, v, df):
    Iu = df.loc[df['userId'] == u, ['movieId']]
    Iv = df.loc[df['userId'] == v, ['movieId']]
    Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')

    nb_rat = df.loc[:, ['movieId', 'rating']].groupby(['movieId']).count()
    
    sum_up = 0
    sum_down_u = 0
    sum_down_v = 0
    for movie in Iuv.movieId.unique() :
        suk = df.loc[(df['userId'] == u) & (df['movieId'] == movie), ['rating_norm']]
        svk = df.loc[(df['userId'] == v) & (df['movieId'] == movie), ['rating_norm']]
        suk, svk = float(suk), float(svk)
        
        sum_up += suk * svk / nb_rat.at[movie, 'rating']
        sum_down_u += suk**2 /  nb_rat.at[movie, 'rating']
        sum_down_v += svk**2 /  nb_rat.at[movie, 'rating']
    return sum_up / math.sqrt(sum_down_u * sum_down_v)

In [37]:
def cor_dis(u, v, df):
    beta = 20
    
    Iu = df.loc[df['userId'] == u, ['movieId']]
    Iv = df.loc[df['userId'] == v, ['movieId']]
    Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
    
    correlation = cor(u, v, df)
    return correlation * min(len(Iuv), beta)/beta

In [18]:
def cor_dis_adj(u, v, df):
    beta = 20
    correlation = cor_adj(u, v, df)
    return correlation * min(len(Iuv), beta)/beta

**Test des taux de corrélation sur les utilisateurs 2 et 3 qui ont 8 films en communs**

In [94]:
u, v = 2, 3
df = ratings_small
Iu = df.loc[df['userId'] == u, ['movieId']]
Iv = df.loc[df['userId'] == v, ['movieId']]
Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
print(Iuv)

print(cor(u, v, df))
print(cor_adj(u, v, df))
print(cor_dis(u, v, df))
# print(cor_dis_adj(u, v, df))

    movieId
27      110
49      296
57      356
64      377
79      527
88      588
91      592
92      593
-0.016060945830838957


TypeError: float() argument must be a string or a number, not 'DataFrame'

In [92]:
Iu = df.loc[df['userId'] == u, ['movieId']]
Iv = df.loc[df['userId'] == v, ['movieId']]
Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')

su = df.loc[(df['userId'] == u) & (df['movieId'].isin(Iuv['movieId']))]
sv = df.loc[(df['userId'] == v) & (df['movieId'].isin(Iuv['movieId']))]


nb_rat = df.loc[:, ['movieId', 'rating']].groupby(['movieId']).count()


sum_up = 0
sum_down_u = 0
sum_down_v = 0
for movie in Iuv.movieId.unique() :
    suk = df.loc[(df['userId'] == u) & (df['movieId'] == movie), ['rating_norm']]
    svk = df.loc[(df['userId'] == v) & (df['movieId'] == movie), ['rating_norm']]

    sum_up += suk * svk / nb_rat.at[movie, 'rating']
    sum_down_u += suk**2 /  nb_rat.at[movie, 'rating']
    sum_down_v += svk**2 /  nb_rat.at[movie, 'rating']

In [156]:
def correlation_matrix(df, cor_fct=cor):
    correlation = []
    users = df.userId.unique()
    couples = []
    for i in range(len(users)):
        u = users[i]
        print('\nuser : ', u)
        for j in range(i + 1, len(users)):
            v = users[j]
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(u, v, df))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    df = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return df

In [140]:
# beware, this cell takes 2 hours to run
cm = correlation_matrix(ratings)


user :  1
	user :  2	user :  3	user :  4	user :  5	user :  6	user :  7	user :  8	user :  9	user :  10	user :  11	user :  12	user :  13	user :  14	user :  15	user :  16	user :  17	user :  18	user :  19	user :  20	user :  21	user :  22	user :  23	user :  24	user :  25	user :  26	user :  27	user :  28	user :  29	user :  30	user :  31	user :  32	user :  33	user :  34	user :  35	user :  36	user :  37	user :  38	user :  39	user :  40	user :  41	user :  42	user :  43

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


	user :  44	user :  45	user :  46	user :  47	user :  48	user :  49	user :  50	user :  51	user :  52	user :  53	user :  54	user :  55	user :  56	user :  57	user :  58	user :  59	user :  60	user :  61	user :  62	user :  63	user :  64	user :  65	user :  66	user :  67	user :  68	user :  69	user :  70	user :  71	user :  72	user :  73	user :  74	user :  75	user :  76	user :  77	user :  78	user :  79	user :  80	user :  81	user :  82	user :  83	user :  84	user :  85	user :  86	user :  87	user :  88	user :  89	user :  90	user :  91	user :  92	user :  93	user :  94	user :  95	user :  96	user :  97	user :  98	user :  99	user :  100	user :  101	user :  102	user :  103	user :  104	user :  105	user :  106	user :  107	user :  108	user :  109	user :  110	user :  111	user :  112	user :  113	user :  114	user :  115	user :  116	user :  117	user :  118	user :  119	user :  120	user :  121	user :  122	user :  123	user :  124	user :  125	user :  126	user :  127	user :  128	user :  129	user :  130	user :  131

	user :  4	user :  5	user :  6	user :  7	user :  8	user :  9	user :  10	user :  11	user :  12	user :  13	user :  14	user :  15	user :  16	user :  17	user :  18	user :  19	user :  20	user :  21	user :  22	user :  23	user :  24	user :  25	user :  26	user :  27	user :  28	user :  29	user :  30	user :  31	user :  32	user :  33	user :  34	user :  35	user :  36	user :  37	user :  38	user :  39	user :  40	user :  41	user :  42	user :  43	user :  44	user :  45	user :  46	user :  47	user :  48	user :  49	user :  50	user :  51	user :  52	user :  53	user :  54	user :  55	user :  56	user :  57	user :  58	user :  59	user :  60	user :  61	user :  62	user :  63	user :  64	user :  65	user :  66	user :  67	user :  68	user :  69	user :  70	user :  71	user :  72	user :  73	user :  74	user :  75	user :  76	user :  77	user :  78	user :  79	user :  80	user :  81	user :  82	user :  83	user :  84	user :  85	user :  86	user :  87	user :  88	user :  89	user :  90	user :  91	user :  92	user :  93	user :  94	user

	user :  6	user :  7	user :  8	user :  9	user :  10	user :  11	user :  12	user :  13	user :  14	user :  15	user :  16	user :  17	user :  18	user :  19	user :  20	user :  21	user :  22	user :  23	user :  24	user :  25	user :  26	user :  27	user :  28	user :  29	user :  30	user :  31	user :  32	user :  33	user :  34	user :  35	user :  36	user :  37	user :  38	user :  39	user :  40	user :  41	user :  42	user :  43	user :  44	user :  45	user :  46	user :  47	user :  48	user :  49	user :  50	user :  51	user :  52	user :  53	user :  54	user :  55	user :  56	user :  57	user :  58	user :  59	user :  60	user :  61	user :  62	user :  63	user :  64	user :  65	user :  66	user :  67	user :  68	user :  69	user :  70	user :  71	user :  72	user :  73	user :  74	user :  75	user :  76	user :  77	user :  78	user :  79	user :  80	user :  81	user :  82	user :  83	user :  84	user :  85	user :  86	user :  87	user :  88	user :  89	user :  90	user :  91	user :  92	user :  93	user :  94	user :  95	user :  96	us

...

	user :  566	user :  567	user :  568	user :  569	user :  570	user :  571	user :  572	user :  573	user :  574	user :  575	user :  576	user :  577	user :  578	user :  579	user :  580	user :  581	user :  582	user :  583	user :  584	user :  585	user :  586	user :  587	user :  588	user :  589	user :  590	user :  591	user :  592	user :  593	user :  594	user :  595	user :  596	user :  597	user :  598	user :  599	user :  600	user :  601	user :  602	user :  603	user :  604	user :  605	user :  606	user :  607	user :  608	user :  609	user :  610	user :  611	user :  612	user :  613	user :  614	user :  615	user :  616	user :  617	user :  618	user :  619	user :  620	user :  621	user :  622	user :  623	user :  624	user :  625	user :  626	user :  627	user :  628	user :  629	user :  630	user :  631	user :  632	user :  633	user :  634	user :  635	user :  636	user :  637	user :  638	user :  639	user :  640	user :  641	user :  642	user :  643	user :  644	user :  645	user :  646	user :  647	user :  648	use

	user :  579	user :  580	user :  581	user :  582	user :  583	user :  584	user :  585	user :  586	user :  587	user :  588	user :  589	user :  590	user :  591	user :  592	user :  593	user :  594	user :  595	user :  596	user :  597	user :  598	user :  599	user :  600	user :  601	user :  602	user :  603	user :  604	user :  605	user :  606	user :  607	user :  608	user :  609	user :  610	user :  611	user :  612	user :  613	user :  614	user :  615	user :  616	user :  617	user :  618	user :  619	user :  620	user :  621	user :  622	user :  623	user :  624	user :  625	user :  626	user :  627	user :  628	user :  629	user :  630	user :  631	user :  632	user :  633	user :  634	user :  635	user :  636	user :  637	user :  638	user :  639	user :  640	user :  641	user :  642	user :  643	user :  644	user :  645	user :  646	user :  647	user :  648	user :  649	user :  650	user :  651	user :  652	user :  653	user :  654	user :  655	user :  656	user :  657	user :  658	user :  659	user :  660	user :  661	use

	user :  594	user :  595	user :  596	user :  597	user :  598	user :  599	user :  600	user :  601	user :  602	user :  603	user :  604	user :  605	user :  606	user :  607	user :  608	user :  609	user :  610	user :  611	user :  612	user :  613	user :  614	user :  615	user :  616	user :  617	user :  618	user :  619	user :  620	user :  621	user :  622	user :  623	user :  624	user :  625	user :  626	user :  627	user :  628	user :  629	user :  630	user :  631	user :  632	user :  633	user :  634	user :  635	user :  636	user :  637	user :  638	user :  639	user :  640	user :  641	user :  642	user :  643	user :  644	user :  645	user :  646	user :  647	user :  648	user :  649	user :  650	user :  651	user :  652	user :  653	user :  654	user :  655	user :  656	user :  657	user :  658	user :  659	user :  660	user :  661	user :  662	user :  663	user :  664	user :  665	user :  666	user :  667	user :  668	user :  669	user :  670	user :  671
user :  594
	user :  595	user :  596	user :  597	user :  598	us

	user :  613	user :  614	user :  615	user :  616	user :  617	user :  618	user :  619	user :  620	user :  621	user :  622	user :  623	user :  624	user :  625	user :  626	user :  627	user :  628	user :  629	user :  630	user :  631	user :  632	user :  633	user :  634	user :  635	user :  636	user :  637	user :  638	user :  639	user :  640	user :  641	user :  642	user :  643	user :  644	user :  645	user :  646	user :  647	user :  648	user :  649	user :  650	user :  651	user :  652	user :  653	user :  654	user :  655	user :  656	user :  657	user :  658	user :  659	user :  660	user :  661	user :  662	user :  663	user :  664	user :  665	user :  666	user :  667	user :  668	user :  669	user :  670	user :  671
user :  613
	user :  614	user :  615	user :  616	user :  617	user :  618	user :  619	user :  620	user :  621	user :  622	user :  623	user :  624	user :  625	user :  626	user :  627	user :  628	user :  629	user :  630	user :  631	user :  632	user :  633	user :  634	user :  635	user :  636	us

	user :  642	user :  643	user :  644	user :  645	user :  646	user :  647	user :  648	user :  649	user :  650	user :  651	user :  652	user :  653	user :  654	user :  655	user :  656	user :  657	user :  658	user :  659	user :  660	user :  661	user :  662	user :  663	user :  664	user :  665	user :  666	user :  667	user :  668	user :  669	user :  670	user :  671
user :  642
	user :  643	user :  644	user :  645	user :  646	user :  647	user :  648	user :  649	user :  650	user :  651	user :  652	user :  653	user :  654	user :  655	user :  656	user :  657	user :  658	user :  659	user :  660	user :  661	user :  662	user :  663	user :  664	user :  665	user :  666	user :  667	user :  668	user :  669	user :  670	user :  671
user :  643
	user :  644	user :  645	user :  646	user :  647	user :  648	user :  649	user :  650	user :  651	user :  652	user :  653	user :  654	user :  655	user :  656	user :  657	user :  658	user :  659	user :  660	user :  661	user :  662	user :  663	user :  664	user :  665	u

In [155]:
test = ratings.loc[ratings['userId'] <= 30]
cm = correlation_matrix(test)


user :  1
	user :  2	user :  3	user :  4	user :  5	user :  6	user :  7	user :  8	user :  9	user :  10	user :  11	user :  12	user :  13	user :  14	user :  15

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


	user :  16	user :  17	user :  18	user :  19	user :  20	user :  21	user :  22	user :  23	user :  24	user :  25	user :  26	user :  27	user :  28	user :  29	user :  30
user :  2
	user :  3	user :  4	user :  5	user :  6	user :  7	user :  8	user :  9	user :  10	user :  11	user :  12	user :  13	user :  14	user :  15	user :  16	user :  17	user :  18	user :  19	user :  20	user :  21	user :  22	user :  23	user :  24	user :  25	user :  26	user :  27	user :  28	user :  29	user :  30
user :  3
	user :  4	user :  5	user :  6	user :  7	user :  8	user :  9	user :  10	user :  11	user :  12	user :  13	user :  14	user :  15	user :  16	user :  17	user :  18	user :  19	user :  20	user :  21	user :  22	user :  23	user :  24	user :  25	user :  26	user :  27	user :  28	user :  29	user :  30
user :  4
	user :  5	user :  6	user :  7	user :  8	user :  9	user :  10	user :  11	user :  12	user :  13	user :  14	user :  15	user :  16	user :  17	user :  18	user :  19	user :  20	user :  21	user :  22	user :  23	user 

In [157]:
print(cm.head(30))

      correlation
u v              
1 2           NaN
  3           NaN
  4      0.042137
  5     -1.000000
  6           NaN
  7     -0.752427
  8           NaN
  9      1.000000
  10          NaN
  11          NaN
  12          NaN
  13          NaN
  14          NaN
  15     0.043773
  16          NaN
  17    -0.972281
  18          NaN
  19     0.053458
  20    -1.000000
  21    -0.478805
  22     0.701197
  23     0.025864
  24          NaN
  25     1.000000
  26     1.000000
  27    -1.000000
  28          NaN
  29          NaN
  30    -0.373096
2 3     -0.016061


In [ ]:
def get_cor(u, v):
    if u > v :
        u, v = v, u
    index = cm.index.values()
    if (u, v)is in index :
        return cm[(u, v)]
    return 0